# ***NOTEBOOK 5: MVP***

## **Carga de los datos**

In [1]:
!pip install google-cloud-storage

In [2]:
!pip install sentence-transformers

  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cus

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ast

*   **Dataset: Recipes**

In [4]:
from google.cloud import storage

# Configura el nombre de tu bucket y archivo
bucket_name = 'mi-bucket-tfg'
file_path = 'datos/df_recipe_aspects.pkl'
local_path = '/tmp/df_recipe_aspects.pkl'

# Conectar a Google Cloud Storage
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_path)

# Cargar el archivo Pickle en un DataFrame
df_recipes = pd.read_pickle(local_path)
df_recipes.head()

,name,description,servings,recipe_id,recipe_grams,recipe_minutes,ingredients_list,ingredients_and_quantities,instructions,keywords,...,fiber_100g,sugar_100g,protein_100g,time_group,avg_rating,review_count,n_ingredients,nutriscore,highlighted_aspects,criticized_aspects
0,Bourbon Chicken,I searched and finally found this recipe on th...,4,45809.0,330.0,35.0,"[boneless chicken breasts, olive oil, garlic c...","[2 lbs boneless chicken breasts, cut into bite...",[1. Editor's Note: Named Bourbon Chicken beca...,"[dinner, chinese, low-carb, chicken, 60-minute...",...,0.09,6.52,15.18,30-40,4.31,2892.0,11,C,"[aabout thick perfection, great thing, good re...",[]
1,Creamy Cajun Chicken Pasta,N'awlin's style of chicken with an updated Alf...,2,39087.0,373.0,25.0,"[boneless skinless chicken breast halves, ling...","[2 boneless skinless chicken breast halves, cu...",[1. Place chicken and Cajun seasoning in a bow...,"[pasta, dinner, cajun, southern, chicken, 30-m...",...,0.67,0.83,10.54,20-30,4.54,1491.0,12,D,"[perfect cooking, true delight, fresh colour, ...",[stuck recipe]
2,"Yes, Virginia There is a Great Meatloaf",Absolutely delicious meatloaf and sauce! Those...,4,54257.0,232.0,80.0,"[ground beef, bread, egg, vidalia onions, tabl...",[1 1/2 lbs ground beef (ground shoulder roast ...,[1. Meatloaf: Combine meat loaf ingredients an...,"[dinner, southern, low-carb, time-to-make, cou...",...,0.17,8.15,15.30,80-90,4.21,1325.0,11,D,"[fond meatloaf, sweet side, straight ketchup, ...",[]
3,Jo Mama's World Famous Spaghetti,My kids will give up a steak dinner for this s...,10,22782.0,285.0,80.0,"[italian sausage, onion, garlic cloves, diced ...","[2 lbs Italian sausage, casings removed (mild ...","[1. In large, heavy stockpot, brown Italian sa...","[dinner, pasta, italian, weeknight, time-to-ma...",...,1.33,4.00,10.46,80-90,4.42,1273.0,16,D,"[great keeper, best sauce, wonderful recipe, g...",[sausage drained]
4,Pancakes,"This is really a great recipe! It is fast, sim...",1,25690.0,407.0,10.0,"[egg, milk, butter, flour, sugar, baking powde...","[1 egg, 3/4 cup milk, 2 tablespoons butter or ...","[1. Beat egg until fluffy., 2. Add milk and me...","[breakfast, 15-minutes-or-less, time-to-make, ...",...,0.10,0.37,0.69,10-20,4.43,1076.0,7,A,"[fantastic recipe, good pancake, good sweet pa...",[]


*   **Dataset: Reviews**

In [5]:
from google.cloud import storage

# Configura el nombre de tu bucket y archivo
bucket_name = 'mi-bucket-tfg'
file_path = 'datos/df_reviews_filtered.pkl'
local_path = '/tmp/df_reviews_filtered.pkl'

# Conectar a Google Cloud Storage
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_path)

# Cargar el archivo Pickle en un DataFrame
df_reviews = pd.read_pickle(local_path)
df_reviews.head()

,review_id,recipe_id,author_id,rating,review,review_date
0,9,4523,2046,2,i think i did something wrong because i could ...,2000
1,14,44,2085,5,An excellent dish.,2000
2,23,4684,2046,5,this is absolutely delicious. i even served i...,2000
3,28,1451,2416,1,Where does the bbq come in,2000
4,55,7762,2369,3,They taste better than they appear.\r\n,2000


*   **Dataset: Embeddings**

In [6]:
# Configura el nombre de tu bucket y archivo
bucket_name = 'mi-bucket-tfg'
file_path = 'datos/df_recipes_embeddings.pkl'
local_path = '/tmp/df_recipes_embeddings.pkl'

# Conectar a Google Cloud Storage
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_path)

# Cargar el archivo Pickle en un DataFrame
df_embeddings = pd.read_pickle(local_path)
df_embeddings.head()

,name,description,servings,recipe_id,recipe_grams,recipe_minutes,ingredients_list,ingredients_and_quantities,instructions,keywords,...,WR,normalized_name,recipe_name_embeddings,similarity,normalized_ingredients_list,ingredients_embeddings,normalized_keywords,keywords_embeddings,normalized_highlighted_aspects,aspects_embeddings
30,Roasted Asparagus,"I got this recipe from ""Simple Vegetarian Plea...",4,50847.0,119.0,25.0,"[asparagus, olive oil, kosher salt]","[1 lb asparagus, 1 1/2 tablespoons olive oil, ...","[1. Preheat oven to 425°F., 2. Cut off the woo...","[side, dinner, vegetarian, low-carb, low-calor...",...,4.559188,roasted asparagus,"[0.70774525, 0.17281596, -0.53876466, 0.269132...",0.394230,"[asparagus, olive oil, kosher salt]","[0.068096995, -0.039195217, -0.66528445, -0.07...","[side, dinner, vegetarian, low carb, low calor...","[0.3077394, 0.32426354, 0.0824065, 0.06405778,...","[wonderful recipe, wonderful second day, perfe...","[-0.20602725, 0.14390379, 0.14365907, -0.11389..."
108,Mexican Stack-Up #RSC,"Ready, Set, Cook! Reynolds Wrap Contest Entry....",6,486261.0,351.0,75.0,"[poblano pepper, reynolds wrap foil, olive oil...","[1 poblano pepper, Reynolds Wrap Foil, 2 table...",[1. Preheat broiler to high broil. Set the ra...,"[dinner, low-carb, mexican, time-to-make, cour...",...,4.531710,mexican stackup rsc,"[-0.7105343, -0.46382508, -0.65078366, 0.13243...",0.507026,"[poblano pepper, reynolds wrap foil, olive oil...","[-0.6319319, -0.24501374, -0.21909977, 0.15086...","[dinner, low carb, mexican, time to make, cour...","[0.27554333, 0.054395784, -0.22202574, 0.00325...","[nice alternative wing, enough amazing making,...","[-0.3275181, -0.072583556, -0.08541173, -0.266..."
46,Roasted Cauliflower &amp; 16 Roasted Cloves of...,"I find it totally amazing, that children and m...",6,106251.0,158.0,40.0,"[cauliflower, garlic cloves, fresh rosemary, s...",[1 large cauliflower (trimmed and cut into bit...,"[1. Mix oil, rosemary, salt, pepper and garlic...","[side, vegetarian, low-carb, low-calorie, vega...",...,4.515852,roasted cauliflower amp roasted cloves of garlic,"[0.12248431, -0.42801306, -0.38706228, -0.1775...",0.520282,"[cauliflower, garlic cloves, fresh rosemary, s...","[-0.2948644, -0.7789556, -0.25072205, -0.25720...","[side, vegetarian, low carb, low calorie, vega...","[0.2790315, 0.1937376, 0.12796554, 0.001535448...","[delicious rita, perfect love, excellent flavo...","[-0.43774918, -0.24240962, 0.28488114, -0.1198..."
7,Kittencal's Moist Cheddar-Garlic Oven Fried Ch...,This is for serious garlic lovers. The chicken...,4,82102.0,185.0,60.0,"[butter, garlic, garlic powder, seasoning salt...","[1/3 cup butter, melted , 2 tablespoons minced...","[1. Preheat oven to 350°F., 2. Butter an 11 x ...","[dinner, chicken, 60-minutes-or-less, time-to-...",...,4.514544,kittencals moist cheddargarlic oven fried chic...,"[0.004746352, -0.12346557, -0.16879883, 0.1019...",0.428067,"[butter, garlic, garlic powder, seasoning salt...","[-0.5912476, 0.18889797, -0.24024855, -0.00016...","[dinner, chicken, 60 minutes or less, time to ...","[0.10633341, -0.2303488, 0.035587464, -0.31038...","[full robust garlic flavor, fine happy meal, f...","[-0.31652376, 0.08892343, -0.16739975, -0.1637..."
38,Roasted Green Beans,"A great alternative to fast-food French fries,...",4,133821.0,154.0,25.0,"[green beans, olive oil, kosher salt, fresh gr...","[2 lbs green beans, 1 or 2 tablespoon olive o...","[1. Preheat oven to 400°F., 2. Wash, dry well,...","[side, lactose-free, vegetarian, snack, lunch,...",...,4.514403,roasted green beans,"[-0.07211356, 0.03044193, -0.5078084, 0.067886...",0.562420,"[green beans, olive oil, kosher salt, fresh gr...","[-0.4692881, -0.05639726, -0.6351077, -0.14723...","[side, lactose free, vegetarian, snack, lunch,...","[0.18054648, 0.15735793, -0.1573779, 0.0844852...","[great frozen green bean, exciting way, awesom...","[-0.7283577, 0.58083975, -0.41203198, -0.07102..."


*   **Dataset: Filtrado Colaborativo**

In [7]:
# Configura el nombre de tu bucket y archivo
bucket_name = 'mi-bucket-tfg'
file_path = 'datos/all_recommendations.csv'
local_path = '/tmp/all_recommendations.csv'

# Conectar a Google Cloud Storage
client = storage.Client()
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(local_path)

# Cargar el archivo Pickle en un DataFrame
df_recommendations = pd.read_csv(local_path)
df_recommendations.head()

,author_id,recipe_id,predicted_rating
0,1533,"[84311, 180011, 327537, 30518, 123839]","[5.960570812225342, 5.832673072814941, 5.79743..."
1,1535,"[128115, 180011, 229112, 327537, 297957]","[5.643883228302002, 5.634768486022949, 5.59416..."
2,1634,"[180011, 264480, 524933, 19698, 20863]","[5.0227837562561035, 4.996372699737549, 4.9559..."
3,1676,"[229112, 180011, 19698, 327537, 84311]","[5.527472019195557, 5.520003795623779, 5.51046..."
4,1792,"[128115, 180011, 84311, 229112, 327537]","[5.48369026184082, 5.411787509918213, 5.353956..."


# **4. Producto Mínimo Viable (MVP)**

Antes de armar el MVP es importante volver a crear las funciones en este notebook:

## **Sistema de Recomendación Basados en Contenido**

### **Busqueda por Receta**

In [8]:
def recomendacion_por_receta(recipe_name, df, model, top_n=3):
    # Generar el embedding para la receta ingresada
    query_embedding = model.encode(recipe_name, convert_to_numpy=True)

    # Calcular similitud coseno entre la consulta y cada receta
    df['similarity'] = df['recipe_name_embeddings'].apply(
        lambda x: cosine_similarity([query_embedding], [x])[0][0]
    )

    # Ordenar por similitud primero y por WR como criterio secundario
    recetas_recomendadas = df.sort_values(by=['similarity', 'WR'], ascending=[False, False]).head(top_n)

    if recetas_recomendadas.empty:
        print("\n↳ No recipes match your query. Please try different names.")
    else:
        print("\nRECIPES:")
        imprimir_recetas(recetas_recomendadas)

### **Busqueda por Ingredientes**

In [9]:
def recomendacion_por_ingredientes_embeddings(ingredientes, df, model, top_n=3):
    # Generar el embedding para los ingredientes ingresados
    query_embedding = model.encode(' '.join(ingredientes), convert_to_numpy=True)

    # Calcular similitud coseno entre la consulta y cada receta
    df['similarity'] = df['ingredients_embeddings'].apply(
        lambda x: cosine_similarity([query_embedding], [x])[0][0] if x is not None else 0
    )

    # Ordenar por similitud de coseno primero y luego por WR
    recetas_recomendadas = df.sort_values(by=['similarity', 'WR'], ascending=[False, False]).head(top_n)

    if recetas_recomendadas.empty:
        print("\n↳ No recipes match your query. Please try different ingredients.")
    else:
        print("\nRECIPES:")
        imprimir_recetas(recetas_recomendadas)

### **Busqueda por Keywords**

In [10]:
def recomendacion_por_keywords(keywords, df, model, top_n=3):
    # Generar el embedding para las keywords ingresadas
    query_embedding = model.encode(' '.join(keywords), convert_to_numpy=True)

    df['similarity'] = df['keywords_embeddings'].apply(
        lambda x: cosine_similarity([query_embedding], [x])[0][0]
    )

    # Ordenar por similitud de coseno primero y luego por WR
    recetas_recomendadas = df.sort_values(by=['similarity', 'WR'], ascending=[False, False]).head(top_n)

    # Verificar si hay recetas recomendadas
    if recetas_recomendadas.empty:
        print("\n↳ No recipes match your query. Please try different keywords.")
    else:
        print("\nRECIPES:")
        imprimir_recetas(recetas_recomendadas)

### **Busqueda por Opiniones de otros Usuarios**

In [11]:
def recomendacion_por_opiniones(aspects, df, model, top_n=3):
    # Verificar si se ingresaron aspectos
    if not aspects:
        print("\n↳ No aspects entered. Please provide at least one aspect.")
        return

    # Generar el embedding para los aspectos ingresados
    query_embedding = model.encode(' '.join(aspects), convert_to_numpy=True)

    df['similarity'] = df['aspects_embeddings'].apply(
        lambda x: cosine_similarity([query_embedding], [x])[0][0]
    )

    # Ordenar por similitud de coseno primero y luego por WR
    recetas_recomendadas = df.sort_values(by=['similarity', 'WR'], ascending=[False, False]).head(top_n)

    # Mostrar los resultados
    if recetas_recomendadas.empty:
        print("\n↳ No recipes match your query. Please try different aspects.")
    else:
        print("\nRECIPES:")
        imprimir_recetas(recetas_recomendadas)

### **Busqueda por Filtros**

**Busqueda por Tiempo de Preparación**

In [12]:
def filtrar_recetas_por_tiempo(df, tiempo_maximo, top_n=3):
    try:
        # Filtrar las recetas que tengan un tiempo de preparación menor o igual al tiempo máximo
        recetas_filtradas = df[df['recipe_minutes'] <= tiempo_maximo]

        # Verificar si hay recetas que cumplan con la condición
        if recetas_filtradas.empty:
            print("\n↳ No recipes match your maximum preparation time.")
        else:
            # Ordenar las recetas filtradas por 'WR' (de mayor a menor)
            recetas_ordenadas = recetas_filtradas.sort_values(by='WR', ascending=False)
            print(f"\nFound {len(recetas_ordenadas)} recipes that meet your criteria.")

            # Llamar a la función imprimir_recetas para mostrar las primeras top_n recetas ordenadas
            return imprimir_recetas(recetas_ordenadas.head(top_n))
    except ValueError:
        print("Please enter a valid number for the maximum preparation time.")

**Busqueda por Cantidad de Porciones**

In [13]:
def filtrar_recetas_por_porciones(df, min_porciones, top_n=3):
    try:
        # Filtrar las recetas que tengan un número de porciones mayor o igual al minimo especificado
        recetas_filtradas = df[df['servings'] >= min_porciones]

        # Verificar si hay recetas que cumplan con la condición
        if recetas_filtradas.empty:
            print("\n↳ No recipes match your minimum servings criteria.")
        else:
            # Ordenar las recetas filtradas por 'WR' (de mayor a menor)
            recetas_ordenadas = recetas_filtradas.sort_values(by=['servings', 'WR'], ascending=[True, False])
            print(f"\nFound {len(recetas_ordenadas)} recipes that meet your criteria.")

            # Llamar a la función imprimir_recetas para mostrar las primeras top_n recetas ordenadas
            return imprimir_recetas(recetas_ordenadas.head(top_n))
    except ValueError:
        print("Please enter a valid number for the minimum servings.")

**Busqueda por Cantidad de Ingredientes**

In [14]:
def filtrar_recetas_por_ingredientes(df, max_ingredientes, top_n=3):
    try:
        # Filtrar las recetas que tengan un número de ingredientes menor o igual al máximo especificado
        recetas_filtradas = df[df['n_ingredients'] <= max_ingredientes]

        # Verificar si hay recetas que cumplan con la condición
        if recetas_filtradas.empty:
            print("\n↳ No recipes match your maximum ingredients criteria.")
        else:
            # Ordenar las recetas filtradas por 'n_ingredients' (ascendente) y luego por 'WR' (descendente)
            recetas_ordenadas = recetas_filtradas.sort_values(by=['n_ingredients', 'WR'], ascending=[False, False])
            print(f"\nFound {len(recetas_ordenadas)} recipes that meet your criteria.")

            # Llamar a la función imprimir_recetas para mostrar las primeras top_n recetas ordenadas
            return imprimir_recetas(recetas_ordenadas.head(top_n))
    except ValueError:
        print("Please enter a valid number for the maximum ingredients.")

**Busqueda por Nutri-Score**

In [15]:
def filtrar_recetas_por_nutriscore(df, nutriscore_deseado, top_n=3):
    try:
        # Convertir el nutriscore deseado a mayúscula (por si el usuario lo escribe en minúsculas)
        nutriscore_deseado = nutriscore_deseado.upper()

        # Verificar que el nutriscore esté dentro de los valores válidos
        if nutriscore_deseado not in ['A', 'B', 'C', 'D', 'E']:
            print("Please enter a valid Nutriscore (A, B, C, D, or E).")
            return

        # Filtrar las recetas que tengan el nutriscore deseado o mejor
        recetas_filtradas = df[df['nutriscore'] <= nutriscore_deseado]

        # Verificar si hay recetas que cumplan con la condición
        if recetas_filtradas.empty:
            print("\n↳ No recipes match your Nutriscore criteria.")
        else:
            # Ordenar las recetas filtradas por 'nutriscore' (ascendente) y luego por 'WR' (descendente)
            recetas_ordenadas = recetas_filtradas.sort_values(by=['nutriscore', 'WR'], ascending=[False, False])
            print(f"\nFound {len(recetas_ordenadas)} recipes that meet your criteria.")

            # Llamar a la función imprimir_recetas para mostrar las primeras top_n recetas ordenadas
            return imprimir_recetas(recetas_ordenadas.head(top_n))
    except ValueError:
        print("Please enter a valid number for the Nutriscore (A, B, C, D, or E).")

### **Filtrado Colaborativo**

In [16]:
def show_user_recommendations(user_id, df_recommendations, df_recipes):
    
    # Asegurar que el user_id sea del mismo tipo que la columna author_id
    if df_recommendations['author_id'].dtype != type(user_id):
        try:
            user_id = df_recommendations['author_id'].dtype.type(user_id)
        except ValueError:
            print(f"Error: El ID del usuario {user_id} no es compatible con los datos.")
            return

    # Filtrar las recomendaciones por el ID del autor (user_id)
    user_recommendations = df_recommendations[df_recommendations["author_id"] == user_id]

    if not user_recommendations.empty:
        # Convertir cadenas en listas reales si es necesario
        try:
            user_recommendations["recipe_id"] = user_recommendations["recipe_id"].apply(
                lambda x: ast.literal_eval(x) if isinstance(x, str) else x
            )
        except (ValueError, SyntaxError):
            print("Error: La columna 'recipe_id' contiene valores no válidos.")
            return

        # Obtener los recipe_id recomendados para este usuario
        recipe_ids = user_recommendations["recipe_id"].explode().tolist()

        # Filtrar df_recipes por los recipe_id obtenidos
        recetas_filtradas = df_recipes[df_recipes["recipe_id"].isin(recipe_ids)]

        # Imprimir las recetas recomendadas
        if not recetas_filtradas.empty:
            print(f"\nRecommendations for user {user_id}:")
            print(imprimir_recetas(recetas_filtradas))
        else:
            print(f"\nNo recipes found for user {user_id}.")
    else:
        print(f"\nNo recommendations found for user {user_id}.")

## **MVP**

In [17]:
def imprimir_recetas(df):
    for index, row in df.iterrows():
        print(f"\n{row['name']}")
        print(f"↳ {row['description']}")
        print(f"\nServings: {row['servings']}")
        print(f"Recipe Grams: {row['recipe_grams']}g")
        print(f"Preparation Time: {int(row['recipe_minutes'])} minutes")
        print(f"Nutriscore: {row['nutriscore']}")

        # Imprimir los ingredientes
        print(f"\nIngredients and Quantities:")
        for ingredient in row['ingredients_and_quantities']:
            print(f"  - {ingredient}")

        # Imprimir las instrucciones
        print(f"\nInstructions:")
        for instruction in row['instructions']:
            print(f"{instruction}")
            
        # Imprimir los highlighted_aspects
        print(f"\nAspectos destacados por otros usuarios:")
        for aspecto in row['highlighted_aspects']:
            print(f"- {aspecto}")
            
        # Imprimir los criticized_aspects
        print(f"\nAspectos criticados por otros usuarios:")
        for aspecto in row['criticized_aspects']:
            print(f"- {aspecto}")

        print("\n" + "-"*100)

In [18]:
def search_recipe():
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    
    print("Select a criterion to search for a recipe:")
    print("1. Recipe Name")
    print("2. Ingredients")
    print("3. Keywords")
    print("4. Other Users Opinions")
    print("5. Preparation Time")
    print("6. Servings")
    print("7. Number of Ingredients")
    print("8. Nutritional Quality (Nutri-Score)")

    option = input("\nPlease enter the number corresponding to your search criterion: ")

    try:
        option = int(option)
        if option < 1 or option > 8:
            raise ValueError
    except ValueError:
        print("\nInvalid input. Please enter a number between 1 and 8.")
        return

    if option == 1: # Search by recipe name
        recipe_usuario = input("\nPlease enter the name of the recipe: ").strip().lower()
        print(f"\nSearching for recipes with the name '{recipe_usuario}' ...")
        recomendacion_por_receta(recipe_usuario, df_embeddings, model)

    elif option == 2: # Search by ingredients
        ingredientes_usuario = input("\nPlease enter ingredients separated by commas: ")
        ingredientes = [ingrediente.strip().lower() for ingrediente in ingredientes_usuario.split(',') if ingrediente.strip()]
        print(f"\nSearching for recipes containing the ingredients '{ingredientes_usuario}' ...")
        recomendacion_por_ingredientes_embeddings(ingredientes, df_embeddings, model)

    elif option == 3: # Search by keywords
        keywords_usuario = input("\nPlease enter keywords separated by commas: ")
        keywords = [keyword.strip().lower() for keyword in keywords_usuario.split(',') if keyword.strip()]
        print(f"\nSearching for recipes with the keywords '{keywords_usuario}' ...")
        recomendacion_por_keywords(keywords, df_embeddings, model)
    
    elif option == 4: # Search other users opinions
        aspects_usuario = input("\nPlease enter relevant aspects separated by commas: ")
        aspects = [aspect.strip().lower() for aspect in aspects_usuario.split(',') if aspect.strip()]
        print(f"\nSearching for recipes with aspects {aspects_usuario} ...")
        recomendacion_por_opiniones(aspects, df_embeddings, model)
    
    elif option == 5: # Search by preparation time
        tiempo_maximo = int(input("\nEnter the maximum preparation time in minutes: "))
        print(f"Searching for recipes with a preparation time of {tiempo_maximo} minutes...")
        filtrar_recetas_por_tiempo(df_embeddings, tiempo_maximo)

    elif option == 6: # Search by servings
        max_porciones = int(input("\nEnter the minimum servings: "))
        print(f"Searching for recipes with {max_porciones} servings...")
        filtrar_recetas_por_porciones(df_embeddings, max_porciones)

    elif option == 7: # Search by number of ingredients
        max_ingredientes = int(input("\nEnter the maximum number of ingredients: "))
        print(f"\nSearching for recipes with {max_ingredientes} ingredients...")
        filtrar_recetas_por_ingredientes(df_embeddings, max_ingredientes)

    elif option == 8: # Search by nutritional quality (Nutri-Score)
        nutriscore_deseado = input("\nEnter the desired Nutriscore (A, B, C, D, or E): ")
        print(f"\nSearching for recipes with Nutri-Score '{nutriscore_deseado}' ...")
        filtrar_recetas_por_nutriscore(df_embeddings, nutriscore_deseado)

In [28]:
search_recipe()

Select a criterion to search for a recipe:
1. Recipe Name
2. Ingredients
3. Keywords
4. Other Users Opinions
5. Preparation Time
6. Servings
7. Number of Ingredients
8. Nutritional Quality (Nutri-Score)



Please enter the number corresponding to your search criterion:  8

Enter the desired Nutriscore (A, B, C, D, or E):  C



Searching for recipes with Nutri-Score 'C' ...

Found 35533 recipes that meet your criteria.

Roasted Asparagus
↳ I got this recipe from "Simple Vegetarian Pleasures" by Jeanne Lemlin. I LOVE asparagus and am always on the lookout for new ways to make it. This is such an easy and elegant way to serve it.  The asparagus gets a slightly nutty flavor after roasting, which makes it even more yummy.

Servings: 4
Recipe Grams: 119.0g
Preparation Time: 25 minutes
Nutriscore: C

Ingredients and Quantities:
  - 1 lb asparagus
  - 1 1/2 tablespoons olive oil
  - 1/2 teaspoon kosher salt (or 1/4 teaspoon regular table salt)

Instructions:
1. Preheat oven to 425°F.
2. Cut off the woody bottom part of the asparagus spears and discard.
3. With a vegetable peeler, peel off the skin on the bottom 2-3 inches of the spears (this keeps the asparagus from being all "stringy" and if you eat asparagus you know what I mean by that).
4. Place asparagus on foil-lined baking sheet and drizzle with olive oil.
5

In [29]:
def search_recipe_authors(user_input, nrecommendations, df_recipes):
    print("\nSelect a criterion to search for a recipe:")
    print("1. Attributes of the recipe")
    print("2. Recipes liked by similar users")

    option = input("\nPlease enter the number corresponding to your search criterion: ")

    try:
        option = int(option)
        if option < 1 or option > 2:
            raise ValueError
    except ValueError:
        print("\nInvalid input. Please enter a valid option.")
        return

    if option == 1:
        search_recipe()

    elif option == 2:
        show_user_recommendations(user_input, df_recommendations, df_recipes)

In [30]:
def recommendation_system():
    # Obtener la lista de autores únicos
    authors = df_reviews['author_id'].unique().tolist()
    
    # Solicitar el ID del usuario
    user_input = input("\nPlease enter your user ID (if you don't have one, leave it blank): ")
    
    # Si el usuario deja el campo vacío, lo consideramos como no tener un ID
    if user_input == "":
        print("\nYou chose not to provide an ID. Searching for recipes.")
        search_recipe()
        return
    
    try:
        user = int(user_input)
    except ValueError:
        print("\nInvalid input. Please enter a valid ID.")
        return
    
    if user in authors:
        search_recipe_authors(user_input, df_recommendations, df_recipes)
    else:
        search_recipe()

Probar con `2525` por ejemplo.

In [31]:
recommendation_system()


Please enter your user ID (if you don't have one, leave it blank):  2525



Select a criterion to search for a recipe:
1. Attributes of the recipe
2. Recipes liked by similar users



Please enter the number corresponding to your search criterion:  2



Recommendations for user 2525:

Chicken Bell Peppers Onions and Mushrooms With Marsala
↳ Cooking Light.

Servings: 4
Recipe Grams: 387.0g
Preparation Time: 30 minutes
Nutriscore: A

Ingredients and Quantities:
  - cooking spray
  - 1 1/2 lbs chicken breast tenders
  - 1 1/2 cups thinly sliced onions
  - 1 cup thinly vertically sliced red bell pepper (about 1 medium)
  - 2 tablespoons olive oil
  - 1/2 teaspoon salt
  - 1/2 teaspoon black pepper
  - 8 ounces mushrooms, sliced (such as shiitake, cremini, and oyster)
  - 3 tablespoons marsala wine

Instructions:
1. Heat a large nonstick skillet over medium-high heat. Coat pan with cooking spray.
2. Add chicken breast tenders to pan; sauté 7 minutes or until the chicken is done. Remove chicken from pan.
3. Add sliced onion and sliced bell pepper to pan; sauté 5 minutes or until onion starts to brown.
4. Add olive oil, salt, black pepper, and presliced exotic mushrooms to pan; sauté 3 minutes or until mushrooms are tender and onion starts 

/tmp/ipykernel_1/2839696803.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recipe_id"] = user_recommendations["recipe_id"].apply(
